In [2]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED     
nomic-embed-text:latest	0a109f422b47	274 MB	11 days ago 	
llama3:8b              	365c0bd3c000	4.7 GB	2 weeks ago 	
llama3.1:8b            	75382d0899df	4.7 GB	3 weeks ago 	
llama3:text            	870a5d02cfaf	4.7 GB	3 weeks ago 	
qwen2:7b               	e0d4e1163c58	4.4 GB	3 weeks ago 	
gemma2:9b              	ff02c3702f32	5.4 GB	3 weeks ago 	
mistral:latest         	2ae6f6dd7a3d	4.1 GB	2 months ago	


In [3]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [4]:
local_model = "qwen2:7b"
llm = ChatOllama(model=local_model)

In [5]:
llm

ChatOllama(model='qwen2:7b')

In [5]:
def summarize_abstractive(text, llm):
    prompt = f"""Write a summary in a narrative style that presents the content in a continuous, flowing manner, focusing on a chronological order of events. The narrative should be smooth without abrupt breaks, covering all relevant points comprehensively, but without delving into deep, specific details. The content should provide a clear overview of the entire event or story to give readers a broad understanding.
    Please read the following news article carefully and follow the instructions below

    {text}

    Instructions:
    - Write each paragraph based on the sequence of events or background, focusing on a continuous storytelling approach.
    - Use \t at the beginning of each paragraph to create indentation.
    - DO NOT leave blank lines between paragraphs. All paragraphs must be continuous with no blank lines.
    - There is NO need to give titles for each paragraph. Focus on telling the story while covering key points.

    IMPORTANT: All your answers must be in Thai. Ensure that the Thai translation accurately conveys the meaning and context of the original content."""

    system_template = f"""You are an expert summarizer with the ability to distill complex information into concise and accurate summaries. Your summaries are clear, coherent, and excellent at identifying key points while preserving the essence of the original text."""

    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(prompt)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    messages = chat_prompt.format_messages(text=text)

    response = llm(messages)
    return response.content

In [6]:
def summarize_extractive(text, llm):
    prompt = f"""Create a summary by selecting and combining key sentences directly from the original text. The summary should present the content in a chronological order, covering all relevant points comprehensively, but without adding any new words or phrases not present in the original text. The extracted sentences should provide a clear overview of the entire event or story to give readers a broad understanding.
    Please read the following news article carefully and follow the instructions below:

    {text}

    Instructions:
    - Use \t at the beginning of each paragraph to create indentation.
    - DO NOT leave blank lines between paragraphs. All paragraphs must be continuous with no blank lines.
    - DO NOT add any new words or phrases. Use ONLY the exact text from the original article.
    - There is NO need to give titles for each paragraph. Focus on presenting the key information while maintaining the original wording.
    - Please DO NOT generate anything else, generate ONLY the summary.

    IMPORTANT: Ensure that you only extract and combine existing sentences from the Thai text. Do not translate or rephrase any content."""

    system_template = f"""You are an expert in extractive summarization with the ability to identify and select the most important sentences from complex texts. Your summaries are coherent compilations of original sentences that accurately represent the key points and overall narrative of the source material."""

    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(prompt)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    messages = chat_prompt.format_messages(text=text)

    response = llm(messages)
    return response.content

In [6]:
import pandas as pd

df_sum = pd.read_csv(
    r'F:\AI\Super AI SS4\Level 3 - INTERN\Dataset\ThEconSum\AIFORTHAI-TextSummarizationCorpus\ThEconSum-random-10-10.csv')

df_sum.shape

(10, 3)

In [32]:
df_sum = pd.read_csv(
    r'F:\AI\Super AI SS4\Level 3 - INTERN\Jupyter Notebook\Evaluate\qwen2-ThEconSum-20.csv')

df_sum.shape

(20, 6)

In [8]:
from tqdm import tqdm

In [9]:
tqdm.pandas()

df_sum['sum_abstractive'] = df_sum['content'].progress_apply(
    lambda x: summarize_abstractive(x, llm)
)

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\Nitro 5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
100%|██████████| 10/10 [46:58<00:00, 281.88s/it]  


In [33]:
tqdm.pandas()

df_sum['sum_extractive'] = df_sum['content'].progress_apply(
    lambda x: summarize_extractive(x, llm)
)

100%|██████████| 20/20 [29:38<00:00, 88.91s/it]


In [36]:
# df_sum.to_csv(
#     r'F:\AI\Super AI SS4\Level 3 - INTERN\Jupyter Notebook\Evaluate\qwen2-ThEconSum-20.csv')
# df_sum.shape

(20, 7)

### combine-dataset

In [7]:
df_sum = pd.read_csv(
    r'F:\AI\Super AI SS4\Level 3 - INTERN\Jupyter Notebook\Evaluate\qwen2-ThEconSum-20.csv')

df_sum.shape

(20, 5)

In [11]:
df_copy = df_sum[['content', 'abstractive', 'sum_abstractive']].copy()
df_copy.columns

Index(['content', 'abstractive', 'sum_abstractive'], dtype='object')

In [10]:
from bert_score import score
from pythainlp.tokenize import word_tokenize
from rouge import Rouge
from tqdm import tqdm

def calculate_bert_score(candidate, reference):
    P, R, F1 = score([candidate], [reference], lang='th', verbose=False)
    bert_score = {
        'precision': float(P[0]),
        'recall': float(R[0]),
        'f1': float(F1[0])
    }
    return bert_score

### BERT Score

In [ ]:
bert_precisions = []
bert_recalls = []
bert_f1_scores = []

for index, row in tqdm(df_copy.iterrows(), total=df_copy.shape[0], desc="BERT Scores ..."):
    candidate = row['sum_extractive']
    reference = row['extractive']

    bert_score = calculate_bert_score(candidate, reference)

    bert_precisions.append(bert_score['precision'])
    bert_recalls.append(bert_score['recall'])
    bert_f1_scores.append(bert_score['f1'])

df_copy['BERT_Precision'] = bert_precisions
df_copy['BERT_Recall'] = bert_recalls
df_copy['BERT_F1'] = bert_f1_scores

### ROUGE Score

In [13]:
def calculate_rouge_scores_for_row(candidate, reference, engine=None):
    if engine:
        hypothesis_tokenized = ' '.join(word_tokenize(candidate, engine=engine))
        reference_tokenized = ' '.join(word_tokenize(reference, engine=engine))
    else:  # ใช้ default
        hypothesis_tokenized = ' '.join(word_tokenize(candidate))
        reference_tokenized = ' '.join(word_tokenize(reference))

    rouge = Rouge()
    scores = rouge.get_scores(hypothesis_tokenized, reference_tokenized)[0]

    rouge1_f1 = scores['rouge-1']['f']
    rouge2_f1 = scores['rouge-2']['f']
    rougel_f1 = scores['rouge-l']['f']

    avg_rouge_score = (rouge1_f1 + rouge2_f1 + rougel_f1) / 3

    return rouge1_f1, rouge2_f1, rougel_f1, avg_rouge_score

def calculate_update_dataframe_ab(df, engine=None):
    rouge1_scores = []
    rouge2_scores = []
    rougel_scores = []
    rouge_avg_scores = []

    engine_label = engine if engine else 'default'

    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Calculating ROUGE Scores with {engine_label}"):
        candidate = row['sum_abstractive']
        reference = row['abstractive']

        rouge1_f1, rouge2_f1, rougel_f1, avg_rouge_score = calculate_rouge_scores_for_row(candidate, reference, engine=engine)

        rouge1_scores.append(rouge1_f1)
        rouge2_scores.append(rouge2_f1)
        rougel_scores.append(rougel_f1)
        rouge_avg_scores.append(avg_rouge_score)

    df[f'ROUGE-1_{engine_label}'] = rouge1_scores
    df[f'ROUGE-2_{engine_label}'] = rouge2_scores
    df[f'ROUGE-L_{engine_label}'] = rougel_scores
    df[f'ROUGE_Score_{engine_label}'] = rouge_avg_scores

    return df

def calculate_update_dataframe_ex(df, engine=None):
    rouge1_scores = []
    rouge2_scores = []
    rougel_scores = []
    rouge_avg_scores = []

    engine_label = engine if engine else 'default'

    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Calculating ROUGE Scores with {engine_label}"):
        candidate = row['sum_extractive']
        reference = row['extractive']

        rouge1_f1, rouge2_f1, rougel_f1, avg_rouge_score = calculate_rouge_scores_for_row(candidate, reference, engine=engine)

        rouge1_scores.append(rouge1_f1)
        rouge2_scores.append(rouge2_f1)
        rougel_scores.append(rougel_f1)
        rouge_avg_scores.append(avg_rouge_score)

    df[f'ROUGE-1_{engine_label}'] = rouge1_scores
    df[f'ROUGE-2_{engine_label}'] = rouge2_scores
    df[f'ROUGE-L_{engine_label}'] = rougel_scores
    df[f'ROUGE_Score_{engine_label}'] = rouge_avg_scores

    return df

In [15]:
df_copy

,content,abstractive,sum_abstractive
0,ประธานบอร์ดแบงก์ชาติ-ปตท.หมดวาระ ก.ค...,\tประธานบอร์ดแบงก์ชาติ-ปตท.หมดวาระ ก.ค.นี้ คลั...,ที่ระบุว่า ตำแหน่งประธานบอร์ดต้องอายุที่ไม่เกิ...
1,อภิวุฒิ พิมลแสงสุริยา เมื่อ...,สำหรับหลักสูตรตลาดหลักทรัพย์ เอ็ม เอ ...,"เมื่อสัปดาห์ก่อน, อภิวุฒิ พิมลแสงสุริยา ได้เดิ..."
2,OR ม้ามืดเข้าคำนวณ FTSE Large Cap รอ...,บริษัทหลักทรัพย์ ทิสโก้ จำกัด ระบุ F...,"ที่เริ่มต้น,บริษัทหลักทรัพย์ ทิสโก้ จำกัด และบ..."
3,กรุงเทพธุรกิจ ตลาดหลักทรัพย์ฯประกาศห...,\tตลาดหลักทรัพย์แห่งประเทศไทย (ตลท.) ประกาศราย...,"ต้นเดือนพฤศจิกายน 2562, ตลาดหลักทรัพย์แห่งประเ..."
4,ไม่ใช่แค่ที่ดินทำเลทองใน กรุงเทพฯ ที...,ล่าสุดที่ดิน 108 ไร่ ในบริเวณสถานีข...,ที่ดินทำเลทองในกรุงเทพฯของ ร.ฟ.ท.-การรถไฟแห่งป...
5,ณัฐณิชา ดอนสุวรรณ กรุงเทพธุ...,คนในแวดวงพลังงานทั้งภาครัฐและเอกชนต่างรอพิสูจน...,ฮอล์อย่างไร\n\nการสำรวจและผลิตปิโตรเลียมรอบใหม...
6,การพลิกฟื้น คุ้งบางกะเจ้า พื้นที่สีเ...,\tการพลิกฟื้น คุ้งบางกะเจ้า พื้นที่สีเขียวสำคั...,นักเรียนและนักศึกษาของคุ้งบางกะเจ้าได้...
7,โค้งสุดท้ายจ่อเปิดประมูลแหล่งก๊าซเอร...,\tกระทรวงพลังงานเสนอคณะกรรมการนโยบายพลังงานแห่...,"ดร.ศิริ จิระพงษ์พันธ์, รัฐมนตรีว่าการก..."
8,นายยุทธนา วิญญูพงศ์พันธุ์ ผู้ช่วยกรร...,\t นายยุทธนา วิญญูพงศ์พันธุ์ ผู้ช่วยกรรมการผู...,"ยุทธนา วิญญูพงศ์พันธุ์, ผู้ช่วยกรรมการผู้จัดกา..."
9,กนอ.ออกทีโออาร์ มาบตาพุดเฟส 3 ต้นพ.ย...,\tนางสาวกฤตยาพร ทัพภะทัต รองผู้ว่าการการนิคมอุ...,ทุกวันนี้ การพัฒนาโครงสร้างพื้นฐานเพื่อรองรับก...


In [ ]:
df_copy.drop(11, inplace=True)

### Abstractive

In [18]:
df_copy_attacut = df_copy.copy()
df_copy_default = df_copy.copy()

df_copy_attacut = calculate_update_dataframe_ab(df_copy_attacut, engine='attacut')
df_copy_default = calculate_update_dataframe_ab(df_copy_default)  # ใช้ default

avg_attacut = df_copy_attacut[[f'ROUGE-1_attacut', f'ROUGE-2_attacut', f'ROUGE-L_attacut', f'ROUGE_Score_attacut']].mean()
avg_default = df_copy_default[[f'ROUGE-1_default', f'ROUGE-2_default', f'ROUGE-L_default', f'ROUGE_Score_default']].mean()

comparison_df = pd.DataFrame({
    'Metric': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE_Score'],
    'Average_Attacut': avg_attacut.values,
    'Average_Default': avg_default.values
})

Calculating ROUGE Scores with default: 100%|██████████| 19/19 [00:00<00:00, 30.49it/s]


In [19]:
comparison_df

,Metric,Average_Attacut,Average_Default
0,ROUGE-1,0.520245,0.471352
1,ROUGE-2,0.214847,0.200800
2,ROUGE-L,0.449339,0.408289
3,ROUGE_Score,0.394811,0.360147


In [24]:
df_copy = df_sum[['content', 'extractive', 'sum_extractive']].copy()
df_copy.columns

Index(['content', 'extractive', 'sum_extractive'], dtype='object')

In [26]:
df_copy.drop(5, inplace=True)

### Extractive

In [27]:
df_copy_attacut = df_copy.copy()
df_copy_default = df_copy.copy()

df_copy_attacut = calculate_update_dataframe_ex(df_copy_attacut, engine='attacut')
df_copy_default = calculate_update_dataframe_ex(df_copy_default)  # ใช้ default

avg_attacut = df_copy_attacut[[f'ROUGE-1_attacut', f'ROUGE-2_attacut', f'ROUGE-L_attacut', f'ROUGE_Score_attacut']].mean()
avg_default = df_copy_default[[f'ROUGE-1_default', f'ROUGE-2_default', f'ROUGE-L_default', f'ROUGE_Score_default']].mean()

comparison_df = pd.DataFrame({
    'Metric': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE_Score'],
    'Average_Attacut': avg_attacut.values,
    'Average_Default': avg_default.values
})

Calculating ROUGE Scores with default: 100%|██████████| 19/19 [00:00<00:00, 36.55it/s]


In [28]:
comparison_df

,Metric,Average_Attacut,Average_Default
0,ROUGE-1,0.560320,0.518507
1,ROUGE-2,0.288590,0.276807
2,ROUGE-L,0.502558,0.478760
3,ROUGE_Score,0.450489,0.424691
